In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns; sns.set();
import matplotlib.pyplot as plt





Data set downloaded from: https://www.kaggle.com/datasets/muhammetvarl/laptop-price

In [2]:
path = "./datasets/laptops/laptop_price.csv"
laptops = pd.read_csv(path, encoding='latin-1')

In [3]:
laptops.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [4]:
laptops.describe()

,laptop_ID,Inches,Price_euros
count,1303.000000,1303.000000,1303.000000
mean,660.155794,15.017191,1123.686992
std,381.172104,1.426304,699.009043
min,1.000000,10.100000,174.000000
25%,331.500000,14.000000,599.000000
50%,659.000000,15.600000,977.000000
75%,990.500000,15.600000,1487.880000
max,1320.000000,18.400000,6099.000000


In [5]:
laptops.columns

Index(['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros'],
      dtype='object')

In [6]:
import re

#converting Weight from fe '13.7kg' (str) into 13.7 (float)
# Function to extract numeric value from string
def extract_numeric(value):
    match = re.search(r'\d+(\.\d+)?', value)
    return float(match.group()) if match else None

# Apply the function to extract numeric values
laptops['weights_numeric'] = laptops['Weight'].apply(extract_numeric)
laptops['weights_numeric']

0       1.37
1       1.34
2       1.86
3       1.83
4       1.37
        ... 
1298    1.80
1299    1.30
1300    1.50
1301    2.19
1302    2.20
Name: weights_numeric, Length: 1303, dtype: float64

In [7]:
laptops = laptops.drop(columns=['weights_numeric'])
laptops

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg,1499.00
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00


So for now I know that I will have to:  
1. Numeric Attributes    
    1. Ram - '8gb' - > 8 (float), DONE
    2. Memory - '128gb' - > 128 (float) - DONE
    3. Weights - '1.37kg' - > 1.37 (float) DONE
    4. Cpu - for now just number of GHz DONE
    5. Screen Resolution -> for shure creating a feature that from 'smth         1240x720' -> goes to number of pixels   DONE
2. Cathegorical Atributes
    1. Memory - into categories: eg SSD, HDD- more like column SSD-values, column HDD-values 0 if none   DONE
    2. Company - definetly
    3. Product - ???
    4. Typename - definetly
    5. Screen Resolution ???
        - maybe touch screen or no - DONE
    6. OpSys definetly 
    7. Cpu ?
        - for now just number of GHz DONE
    8. Gpu Maybe just AMD vs Intel vs other - DONE 
    

In [8]:
# split into num and cathegorical atributes and then create transformations using pipelines

def extract_numeric(value):
    match = re.search(r'\d+(\.\d+)?', value)
    return float(match.group()) if match else None


    

def trs_laptopsAttr_toNum(data):
    data['Weight'] = data['Weight'].apply(extract_numeric) #tests '1.37kg' -> 1.37
    data['Ram'] = data['Ram'].apply(extract_numeric) #tests: '8GB' -> 8
    return data



In [9]:
data2 = laptops.copy()

In [10]:
data2

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg,1499.00
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00


In [23]:
from laptop_features_extractor import take_size_type_fromMemory
from laptop_features_extractor import take_Ghz_fromCPU, take_nrofPixels_fromScreenResolution

def memory_extractor(memory_column):
    total_memory = np.zeros(memory_column.size)
    SSD = np.zeros(memory_column.size)
    HDD = np.zeros(memory_column.size)
    Other = np.zeros(memory_column.size)
    for i,mem in enumerate(memory_column):
        memory_info = take_size_type_fromMemory(mem)
        #read total memory
        if len(memory_info) == 4:
            memmory_size = memory_info[0] + memory_info[2]
        else:
            memory_size = memory_info[0]
        total_memory[i] = memory_size
        if 'SSD' in mem:
            SSD[i] = 1
        if 'HDD' in mem:
            HDD[i] = 1
        if 'Other' in mem:
            Other[i] = 1
    
    return {"Memory size":total_memory} ,{'SSD': SSD}, {'HDD':HDD}, {'Other': Other}


def cpu_extractor(cpu_column):
    nrs_of_GHz = np.zeros(cpu_column.size)
    for i, word in enumerate(cpu_column):
        nrGHz = take_Ghz_fromCPU(word)
        nrs_of_GHz[i] = nrGHz
    return {"nr_GHz": nrs_of_GHz}

def screenResolution_extractor(screenResolution_column):
    pixels = np.zeros(screenResolution_column.size)
    for i, word in enumerate(screenResolution_column):
        nrpixel = take_nrofPixels_fromScreenResolution(word)
        pixels[i] = nrpixel
    return {"nr_pixels": pixels}


#Here you should test wether the data you are using is really a gpu colum,s
def Gpu_company_extractor(Gpu_column):
    Intel = np.zeros(Gpu_column.size)
    AMD = np.zeros(Gpu_column.size)
    Nvidia = np.zeros(Gpu_column.size)
    Other  = np.zeros(Gpu_column.size)
    for i, data in enumerate(Gpu_column):
        if 'Intel' in data:
            Intel[i] = 1
        elif 'AMD' in data:
            AMD[i] = 1
        elif 'Nvidia' in data:
            Nvidia[i] = 1
        else:
            Other[i] = 1
    return {'Intel': Intel}, {'AMD': AMD}, {'Nvidia': Nvidia}, {'Other_GPU_company': Other}

def ScreenResolution_touchscreen_extractor(screenResolution_column):
    touchScreen = np.zeros(screenResolution_column.size)
    for i, data in enumerate(screenResolution_column):
        if 'Touchscreen' in data:
            touchScreen[i] = 1
    return {'touchscreen':touchScreen}
        

In [32]:
print("How memory_extractor works: ")
memory = data2['Memory'].values[:]
print(memory_extractor(memory))

print("\nHow cpu_exractor works")
cpu = data2['Cpu'].values[:]
print(cpu_extractor(cpu))

print("\nHow screenResolution_extractor works")
pixels = data2['ScreenResolution'].values[:]
print(screenResolution_extractor(pixels))

print("\nHow Gpu_company_extractor works")
gpu = data2['Gpu'].values[:]
print(Gpu_company_extractor(gpu))

print("\nHow ScreenResolution_touchscreen_extractor works")
scr = data2['ScreenResolution'].values[:]
print(ScreenResolution_touchscreen_extractor(scr))

How memory_extractor works: 
({'Memory size': array([ 128.,  128.,  256., ...,   64., 1024.,  500.])}, {'SSD': array([1., 0., 1., ..., 0., 0., 0.])}, {'HDD': array([0., 0., 0., ..., 0., 1., 1.])}, {'Other': array([0., 0., 0., ..., 0., 0., 0.])})

How cpu_exractor works
{'nr_GHz': array([2.3, 1.8, 2.5, ..., 1.6, 2.5, 1.6])}

How screenResolution_extractor works
{'nr_pixels': array([4096000., 1296000., 2073600., ..., 1049088., 1049088., 1049088.])}

How Gpu_company_extractor works
({'Intel': array([1., 1., 1., ..., 1., 0., 1.])}, {'AMD': array([0., 0., 0., ..., 0., 1., 0.])}, {'Nvidia': array([0., 0., 0., ..., 0., 0., 0.])}, {'Other_GPU_company': array([0., 0., 0., ..., 0., 0., 0.])})

How ScreenResolution_touchscreen_extractor works
{'touchscreen': array([0., 0., 0., ..., 0., 0., 0.])}


In [25]:
'Touchscreen' in 'PS Panel Full HD / Touchscreen 1920x1080'

True

In [ ]:
data2['Gpu'].value_counts()

In [ ]:
np.array([1,2])**2

In [ ]:
data = '1366x768'

print(data.split(' ')[-1].split('x'))